In [163]:
%run Solver.ipynb

In [164]:
# Position format: ("xxoo-----", remoteness)
GAMENAME = 'TIC_TAC_TOE'
STARTING_POS = ("---------", 0)
USE_SYMMETRY = True

In [165]:
def DoMove(position, move):
    pos = getValue(position)
    if pos.count('x') > pos.count('o'):
        pos = list(pos)
        pos[move] = 'o'
        pos = "".join(pos)
    else:
        pos = list(pos)
        pos[move] = 'x'
        pos = "".join(pos)
    return (pos, getRemoteness(position))

In [166]:
def GenerateMoves(position): 
    return [i for i, char in enumerate(getValue(position)) if char == '-']

In [167]:
def triple_equal(string, a, b, c):
    return string[a] != '-' and \
string[a] == string[b] and \
string[b] == string[c]

def haspattern(position):
    pos = getValue(position)
    return triple_equal(pos,0,1,2) or \
triple_equal(pos,3,4,5) or triple_equal(pos,6,7,8) \
or triple_equal(pos,0,3,6) or triple_equal(pos,1,4,7) or \
triple_equal(pos,2,5,8) or \
triple_equal(pos,0,4,8) or triple_equal(pos,2,4,6)

def modify_str(string, index, char):
    new = list(string)
    new[index] = char
    

In [168]:
def PrimitiveValue(position):
    if haspattern(position):
        return ("lose", 0)
    elif '-' not in getValue(position):
        return ("tie", 0)
    else:
        return "not_primitive"

In [169]:
def getValue(position):
    return position[0]

def getRemoteness(position):
    return position[1]

In [170]:
#  dict : {position_str: (result, remoteness)}
# position: ("xxoo-----", remoteness)
all_dis_pos = {}
def generate_all_distinct_pos(position):
    # pos: (result, remoteness)
    cononical = exist(position[0], all_dis_pos)
    if cononical:
        return all_dis_pos.get(cononical)
    else:
        pos = PrimitiveValue(position)
        if pos == 'not_primitive':
            hasLose = False
            hasTie = False
            # remoteness of winning children
            ChildWinRemotesArr = []
            # remoteness of tie children
            ChildTieRemotesArr = []
            # remoteness of losing children
            ChildLoseRemotesArr = []
            for move in GenerateMoves(position):
                result = generate_all_distinct_pos(DoMove(position, move))
                if result[0] == 'lose':
                    ChildLoseRemotesArr.append(result[1])
                    hasLose = True
                elif result[0] == 'tie':
                    ChildTieRemotesArr.append(result[1])
                    hasTie = True
                else:
                    ChildWinRemotesArr.append(result[1])
            if hasLose:
                # Player wants to win ASAP
                all_dis_pos.update({position[0]: ("win", min(ChildLoseRemotesArr) + 1)})
                return ("win", min(ChildLoseRemotesArr) + 1)
            elif hasTie:
                # Player wants to tie As slow as possible
                all_dis_pos.update({position[0]: ("tie", max(ChildTieRemotesArr) + 1)})
                return ("tie", max(ChildTieRemotesArr) + 1)
            else:
                # Player wants to lose As slow as possible
                all_dis_pos.update({position[0]: ("lose", max(ChildWinRemotesArr) + 1)})
                return ("lose", max(ChildWinRemotesArr) + 1)    
        # it's a primitive tie/lose
        else:
            all_dis_pos.update({position[0]: pos})
            return pos

In [171]:
# These two transformation functions return an array of duplicate positions
def rotation(position_str):
    retval = []
    retval.append(position_str)
    orig = list(position_str)
    ninty = [orig[6], orig[3], orig[0], orig[7], orig[4], orig[1], orig[8], orig[5], orig[2]]
    retval.append("".join(ninty))
    one_eighty = [orig[8], orig[7], orig[6], orig[5], orig[4], orig[3], orig[2], orig[1], orig[0]]
    retval.append("".join(one_eighty))
    two_seventy = [orig[2], orig[5], orig[8], orig[1], orig[4], orig[7], orig[0], orig[3], orig[6]]
    retval.append("".join(two_seventy))
    return retval

def reflect(position_str):
    retval = []
    orig = list(position_str)
    # flip by the horizontal line 
    horizontal = [orig[6], orig[7], orig[8], orig[3], orig[4], orig[5], orig[0], orig[1], orig[2]]
    retval.append("".join(horizontal))
    vertical = [orig[2], orig[1], orig[0], orig[5], orig[4], orig[3], orig[8], orig[7], orig[6]]
    retval.append("".join(vertical))
    upper_left_lower_right = [orig[0], orig[3], orig[6], orig[1], orig[4], orig[7], orig[2], orig[5], orig[8]]
    retval.append("".join(upper_left_lower_right))
    upper_right_lower_left = [orig[8], orig[5], orig[2], orig[7], orig[4], orig[1], orig[6], orig[3], orig[0]]
    retval.append("".join(upper_right_lower_left))
    return retval


In [177]:
# argument position_str is a length-9 string
def exist(position_str, dictionary):
    rotations = rotation(position_str)
    for e in rotations:
        if e in dictionary:
            return e
    reflects = reflect(position_str)
    for e in reflects:
        if e in dictionary:
            return e
    return None

In [178]:
if USE_SYMMETRY:
    generate_all_distinct_pos(STARTING_POS)

In [179]:
import collections

In [189]:
%%capture cap

if USE_SYMMETRY:
    print('-----------Remoteness Output-----------WITH SYMMETRY')
    all_remoteness = [[],[],[],[],[],[],[],[],[],[]]
    symmetry_total = [0,0,0,0]
    for i in range(10):
        all_remoteness[i] = [0,0,0,0]
    for e in list(all_dis_pos.values()):
        if e[0] == 'win':
            all_remoteness[e[1]][0] += 1
            symmetry_total[0] += 1
        elif e[0] == 'lose':
            all_remoteness[e[1]][1] += 1
            symmetry_total[1] += 1
        elif e[0] == 'tie':
            all_remoteness[e[1]][2] += 1
            symmetry_total[2] += 1
        all_remoteness[e[1]][3] += 1
        symmetry_total[3] += 1
    for i in range(9, -1, -1):
        print(i, '  ', all_remoteness[i])
    print('Total', symmetry_total)

    
else:
    result = Solve(STARTING_POS)
    print('-----------Vanilla Output-----------')
    print("result is a " + result[0] + '!')

    losses = len(lose_s)
    wins = len(win_s)
    ties = len(tie_s)
    pr_losses = len(lose_primitive_s)
    pr_wins = len(win_primitive_s)
    pr_ties = len(tie_primitive_s)

    total_losses = losses + pr_losses
    total_wins = wins + pr_wins
    total_ties = ties + pr_ties
    total = total_losses + total_wins + total_ties
    pr_total = pr_losses + pr_wins + pr_ties

    print("Lose: ", total_losses, ' (', pr_losses, ' primitive)')
    print("Win: ", total_wins, ' (', pr_wins, ' primitive)')
    print("Tie: ", total_ties, ' (', pr_ties, ' primitive)')
    print("Total: ", total, ' (', pr_total, ' primitive)')

    print('-----------------------------------')
    print('-----------Remoteness Output-----------WITHOUT SYMMETRY')

    analysis = {}
    total = [0, 0, 0, 0]
    print('RM     ', 'W ', 'L ', 'T', 'Total')
    print('---------------------')

    # update analysis dict using remoteness as key and array of result as count
    # {remoteness: [#win, #lose, #tie, #total]}
    def update(target_dictionary, source_set, index, total_array):
        for pos in source_set:
            rns = getRemoteness(pos)
            if rns not in target_dictionary:
                target_dictionary.update({rns: [0,0,0,0]})
            count_ls = target_dictionary.get(rns)
            count_ls[index] += 1
            total[index] += 1
            count_ls[3] += 1
            total[3] += 1
            target_dictionary.update({rns: count_ls})

    update(analysis, win_s, 0, total)
    update(analysis, win_primitive_s, 0, total)
    update(analysis, lose_s, 1, total)
    update(analysis, lose_primitive_s, 1, total)
    update(analysis, tie_s, 2, total)
    update(analysis, tie_primitive_s, 2, total)

    # offset = min(analysis.keys())
    od = collections.OrderedDict(sorted(analysis.items(),reverse=True))
    for remote, count_ls in od.items(): print(remote, '    ', count_ls)

    print('---------------------')
    print('Total ', total)

with open(GAMENAME + '_symmetry_output.txt', 'w') as f:
    f.write(cap.stdout)